In [1]:
# Example: https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html

import tensorflow as  tf

import os
ROOT = '/mnt/Work/Projects/Tensorflow-examples/Tensorflow-Datasets_Estimator'
DATAPATH = os.path.join(ROOT, 'data')
FILE = os.path.join(DATAPATH, 'iris_training.csv')


# Create a list to be used in feature columns:
FEATURE_NAMES = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth']   

/home/navar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/navar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Estimators requires an input_fn()

def input_fn(file_path, shuffle=False, repeat_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, record_defaults=[[0.]]*5)
        label = parsed_line[-1:]
        features = parsed_line[:-1]
        return dict(zip(FEATURE_NAMES, features)), label

    # Creates an instace of the TextLineDataset class
    dataset = (tf.data.TextLineDataset(file_path).skip(1).map(decode_csv))
    
    if shuffle:
       # Randomizes input using a window of 256 elements (read into memory)
       dataset = dataset.shuffle(buffer_size=256)
        
    dataset = dataset.repeat(repeat_count) # Repeats dataset this # times
    dataset = dataset.batch(8)  # Batch size to use
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    
    return batch_features, batch_labels
    

In [3]:
next_batch = input_fn(FILE, True) # Will return 32 random elements

with tf.Session() as sess:
    first_batch = sess.run(next_batch)
    
print(first_batch)

({'SepalLength': array([4.8, 5.6, 5.2, 4.4, 4.9, 6.7, 6. , 6.1], dtype=float32), 'SepalWidth': array([3. , 2.7, 2.7, 3. , 3.1, 3. , 2.2, 2.8], dtype=float32), 'PetalLength': array([1.4, 4.2, 3.9, 1.3, 1.5, 5. , 5. , 4. ], dtype=float32), 'PetalWidth': array([0.3, 1.3, 1.4, 0.2, 0.1, 1.7, 1.5, 1.3], dtype=float32)}, array([[0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [2.],
       [1.]], dtype=float32))
